<a href="https://colab.research.google.com/github/susrithac/Data-Warehouse-and-Data-Mining-/blob/main/DWDM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files   
uploaded = files.upload()

Saving bank.csv to bank.csv


In [ ]:
# Loading all required packages
# If any of it fails, do not panic. Just install it using "pip3 install <package_name>" or by using conda install package_name
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

#from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression,LinearRegression

In [144]:
df=pd.read_csv('https://raw.githubusercontent.com/susrithac/Data-Warehouse-and-Data-Mining-/main/bank.csv',sep=';')
df.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
5,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no
6,36,self-employed,married,tertiary,no,307,yes,no,cellular,14,may,341,1,330,2,other,no
7,39,technician,married,secondary,no,147,yes,no,cellular,6,may,151,2,-1,0,unknown,no
8,41,entrepreneur,married,tertiary,no,221,yes,no,unknown,14,may,57,2,-1,0,unknown,no
9,43,services,married,primary,no,-88,yes,yes,cellular,17,apr,313,1,147,2,failure,no


In [ ]:
df.dtypes

# bank client data:
   1 - age (numeric)  
   2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services")   
   3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)  
   4 - education (categorical: "unknown","secondary","primary","tertiary")  
   5 - default: has credit in default? (binary: "yes","no")  
   6 - balance: average yearly balance, in euros (numeric)   
   7 - housing: has housing loan? (binary: "yes","no")  
   8 - loan: has personal loan? (binary: "yes","no")   
   # related with the last contact of  the current campaign:
   9 - contact: contact communication type (categorical: "unknown","telephone","cellular")   
  10 - day: last contact day of the month (numeric)  
  11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")  
  12 - duration: last contact duration, in seconds (numeric)  
   # other attributes:
  13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)  
  14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)  
  15 - previous: number of contacts performed before this campaign and for this client (numeric)  
  16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")  

  Output variable (desired target):  
  17 - y - has the client subscribed a term deposit? (binary: "yes","no")  

In [ ]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [ ]:
print(df.info())
print(df.isnull().sum())

In [ ]:
# first question
for i in df.columns:
    display(df[i].value_counts().head(10))

Without any data preprocessing we are going to fit the data into one of the machine learing alogorithm  
Lets deploy Linear Regression which is very well known and simplest algorithm

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
X=df.drop(['y'],axis=1)
y = df['y']
X_train,X_test , y_train, y_test =train_test_split(X,y,test_size=0.2)
ln = linear_model.LinearRegression()
ln.fit(X_train,y_train)

ValueError: ignored

We simply applied a machine learning algorithm without any preprocessing.. but there is no nan values in the dataset so it should work right?
Well not exactly, Machine learning algorithms doesn't work well with categorical values, if you  plan on doing  predictions while there are categorical columns present in the dataset, make sure that those columns are replaced with a number. Like yes/no - 1/0  or male/female - 1/2 etc. 

We can do this easily by using encoding. Encoding can be done in different ways. So lets explore few of them..


In [145]:
#Lets convert all the columns that are object datatype into category since we have few different values for each category
df['job'] = df.job.astype('category')
df['marital'] = df.marital.astype('category')
df['education'] = df.education.astype('category')
df['default'] = df.default.astype('category')
df['housing'] = df.housing.astype('category')
df['loan'] = df.loan.astype('category')
df['contact'] = df.contact.astype('category')
df['month'] = df.month.astype('category')
df['poutcome'] = df.poutcome.astype('category')
df['y'] = df.y.astype('category')

#### By replacing 

In [146]:
# y is our predicting column so we are replacing yes with 1 , no with 0
df['y'] = df['y'].apply(lambda x: 0 if x.strip()=='no' else 1)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [147]:
df['job'] = df['job'].replace('unknown',np.nan)
print(df.job.isnull().sum())

38


Job column has 38 nan or unknown values, since number of nan values is less compared to number of instances(4599) lets just replace those nan values with most frequent value in the job.

In [148]:
#we can use fillna() to fill all the na values with desired input
df['job'] = df['job'].fillna(df['job'].value_counts().idxmax())
print(df.job.isnull().sum())
df.head()

0


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [150]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df["job"] = ord_enc.fit_transform(df[["job"]])
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10.0,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,7.0,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,4.0,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,4.0,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,1.0,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


The column contact has three categorical values namely cellular, telephone and unknown. Since contact most likely to be either one of them so I am using forward fill to fill all the unknown values.

In [151]:
df['contact'] = df['contact'].replace('unknown',np.nan)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10.0,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,7.0,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,4.0,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,4.0,married,tertiary,no,1476,yes,yes,NaN,3,jun,199,4,-1,0,unknown,0
4,59,1.0,married,secondary,no,0,yes,no,NaN,5,may,226,1,-1,0,unknown,0


In [152]:
#Filling all the nan values usinh forward fill
df['contact'] = df['contact'].ffill(axis = None)
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10.0,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,7.0,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,4.0,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,4.0,married,tertiary,no,1476,yes,yes,cellular,3,jun,199,4,-1,0,unknown,0
4,59,1.0,married,secondary,no,0,yes,no,cellular,5,may,226,1,-1,0,unknown,0


In [153]:
print(df.contact.value_counts())

cellular     4105
telephone     416
Name: contact, dtype: int64


In [154]:
df = pd.get_dummies(df,columns=['marital','default','housing','loan','contact'],prefix=['marital','default','housing','loan','contact'])
df.head()

,age,job,education,balance,day,month,duration,campaign,pdays,previous,poutcome,y,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone
0,30,10.0,primary,1787,19,oct,79,1,-1,0,unknown,0,0,1,0,1,0,1,0,1,0,1,0
1,33,7.0,secondary,4789,11,may,220,1,339,4,failure,0,0,1,0,1,0,0,1,0,1,1,0
2,35,4.0,tertiary,1350,16,apr,185,1,330,1,failure,0,0,0,1,1,0,0,1,1,0,1,0
3,30,4.0,tertiary,1476,3,jun,199,4,-1,0,unknown,0,0,1,0,1,0,0,1,0,1,1,0
4,59,1.0,secondary,0,5,may,226,1,-1,0,unknown,0,0,1,0,1,0,0,1,1,0,1,0


In [155]:
#Doing encoding with Label Encoder
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df['poutcome'] = lb_make.fit_transform(df['poutcome'])

df.poutcome.value_counts()

3    3705
0     490
1     197
2     129
Name: poutcome, dtype: int64

In [156]:
#similar to the poutcome we have applied the label encoder to the month column
df_month = df.copy()
lb_make = LabelEncoder()
df_month['month'] = lb_make.fit_transform(df_month['month'])

df_month.month.value_counts() #Results in appending a new column to df

8     1398
5      706
1      633
6      531
9      389
0      293
3      222
4      148
10      80
11      52
7       49
2       20
Name: month, dtype: int64

But can you see there's a misconception in the encoding??  
yes, it's the order. The values that are assigned to the months are not in order,well it is easy to understand if values of months are sorted. So lets use mapping in this context.

In [157]:
df.month.value_counts()

may    1398
jul     706
aug     633
jun     531
nov     389
apr     293
feb     222
jan     148
oct      80
sep      52
mar      49
dec      20
Name: month, dtype: int64

In [158]:
month_mapping ={'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,"jul":7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
df['month']=df.month.map(month_mapping)
df.month.value_counts()

5     1398
7      706
8      633
6      531
11     389
4      293
2      222
1      148
10      80
9       52
3       49
12      20
Name: month, dtype: int64

In [161]:
df.education.value_counts()

secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64

The number of unknown values in education column is rather low so we changed them into nan values and dropped the rows with nan values.

In [ ]:
df['education'] = df['education'].replace('unknown',np.nan)
df= df.dropna(axis=0)

In [169]:
df["education"] = ord_enc.fit_transform(df[["education"]])
df.head()

,age,job,education,balance,day,month,duration,campaign,pdays,previous,poutcome,y,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone
0,30,10.0,0.0,1787,19,10,79,1,-1,0,3,0,0,1,0,1,0,1,0,1,0,1,0
1,33,7.0,1.0,4789,11,5,220,1,339,4,0,0,0,1,0,1,0,0,1,0,1,1,0
2,35,4.0,2.0,1350,16,4,185,1,330,1,0,0,0,0,1,1,0,0,1,1,0,1,0
3,30,4.0,2.0,1476,3,6,199,4,-1,0,3,0,0,1,0,1,0,0,1,0,1,1,0
4,59,1.0,1.0,0,5,5,226,1,-1,0,3,0,0,1,0,1,0,0,1,1,0,1,0


In [159]:
df.head(10)

,age,job,education,balance,day,month,duration,campaign,pdays,previous,poutcome,y,marital_divorced,marital_married,marital_single,default_no,default_yes,housing_no,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone
0,30,10.0,primary,1787,19,10,79,1,-1,0,3,0,0,1,0,1,0,1,0,1,0,1,0
1,33,7.0,secondary,4789,11,5,220,1,339,4,0,0,0,1,0,1,0,0,1,0,1,1,0
2,35,4.0,tertiary,1350,16,4,185,1,330,1,0,0,0,0,1,1,0,0,1,1,0,1,0
3,30,4.0,tertiary,1476,3,6,199,4,-1,0,3,0,0,1,0,1,0,0,1,0,1,1,0
4,59,1.0,secondary,0,5,5,226,1,-1,0,3,0,0,1,0,1,0,0,1,1,0,1,0
5,35,4.0,tertiary,747,23,2,141,2,176,3,0,0,0,0,1,1,0,1,0,1,0,1,0
6,36,6.0,tertiary,307,14,5,341,1,330,2,1,0,0,1,0,1,0,0,1,1,0,1,0
7,39,9.0,secondary,147,6,5,151,2,-1,0,3,0,0,1,0,1,0,0,1,1,0,1,0
8,41,2.0,tertiary,221,14,5,57,2,-1,0,3,0,0,1,0,1,0,0,1,1,0,1,0
9,43,7.0,primary,-88,17,4,313,1,147,2,0,0,0,1,0,1,0,0,1,0,1,1,0


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   age                4521 non-null   int64   
 1   job                4521 non-null   float64 
 2   education          4521 non-null   category
 3   balance            4521 non-null   int64   
 4   day                4521 non-null   int64   
 5   month              4521 non-null   category
 6   duration           4521 non-null   int64   
 7   campaign           4521 non-null   int64   
 8   pdays              4521 non-null   int64   
 9   previous           4521 non-null   int64   
 10  poutcome           4521 non-null   int64   
 11  y                  4521 non-null   category
 12  marital_divorced   4521 non-null   uint8   
 13  marital_married    4521 non-null   uint8   
 14  marital_single     4521 non-null   uint8   
 15  default_no         4521 non-null   uint8   
 16  defaul

In [164]:
print(df.isnull().sum())

age                  0
job                  0
education            0
balance              0
day                  0
month                0
duration             0
campaign             0
pdays                0
previous             0
poutcome             0
y                    0
marital_divorced     0
marital_married      0
marital_single       0
default_no           0
default_yes          0
housing_no           0
housing_yes          0
loan_no              0
loan_yes             0
contact_cellular     0
contact_telephone    0
dtype: int64


Lets check the accuracy by deploying random forest method

In [170]:
X = df.drop('y',axis=1)
y=df['y']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [171]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [172]:
rf_clf = RandomForestClassifier(n_estimators=1000,
                            bootstrap=True, oob_score=True,
                            n_jobs=-1, random_state=42)
rf_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [173]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        res = clf.predict(X_train)
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, 
                                                                res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, 
                                                                            res)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, 
                                                                  res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                      lb.transform(res))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("\nTest Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, 
                                                                res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, 
                                                                            res_test)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, 
                                                                  res_test)))   
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), 
                                                      lb.transform(res_test))))
        

In [174]:
print_score(rf_clf, X_train, X_test, y_train, y_test, train=True)
print_score(rf_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2795
           1       1.00      1.00      1.00       369

    accuracy                           1.00      3164
   macro avg       1.00      1.00      1.00      3164
weighted avg       1.00      1.00      1.00      3164


Confusion Matrix: 
 [[2795    0]
 [   0  369]]

ROC AUC: 1.0000

Average Accuracy: 	 0.8979
Accuracy SD: 		 0.0086

Test Result:

accuracy score: 0.8998

Classification Report: 
               precision    recall  f1-score   support

           0       0.91      0.98      0.95      1205
           1       0.62      0.28      0.38       152

    accuracy                           0.90      1357
   macro avg       0.77      0.63      0.66      1357
weighted avg       0.88      0.90      0.88      1357


Confusion Matrix: 
 [[1179   26]
 [ 110   42]]

ROC AUC: 0.6274



In [ ]:
#To download and view
from google.colab import files
df1.to_csv('filename1.csv')
files.download("filename1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import random
J = random.sample(range(4521), 295)

In [ ]:
#Creating NaN values in age col
A = []
for i in J:
  A.append(df1['age'][i]) #Having the original values for future analysis
  df1['age'][i] = np.nan
df1['age']


0       30.0
1       33.0
2       35.0
3       30.0
4       59.0
        ... 
4516    33.0
4517    57.0
4518    57.0
4519    28.0
4520    44.0
Name: age, Length: 4521, dtype: float64

In [ ]:
for i in df1['age']:
  print(i)

In [ ]:
df1['balance']

0       1787
1       4789
2       1350
3       1476
4          0
        ... 
4516    -333
4517   -3313
4518     295
4519    1137
4520    1136
Name: balance, Length: 4521, dtype: int64

In [ ]:
#Creating NaN values in balance for all -ve values
for i in range(len(df1['balance'])):
  if(df1['balance'][i] < 0):
    df1['balance'][i] = np.nan
df1['balance'] 

0       1787.0
1       4789.0
2       1350.0
3       1476.0
4          0.0
         ...  
4516       NaN
4517       NaN
4518     295.0
4519    1137.0
4520    1136.0
Name: balance, Length: 4521, dtype: float64

In [ ]:
#Creating random values in duration
J2 = random.sample(range(4521), 291)
B = []
for i in J2:
  B.append(df1['duration'][i])
  df1['duration'][i] = np.nan
df1['duration']

0        79.0
1       220.0
2       185.0
3       199.0
4       226.0
        ...  
4516    329.0
4517    153.0
4518    151.0
4519    129.0
4520    345.0
Name: duration, Length: 4521, dtype: float64

In [ ]:
A #Has original values which are tampered to np.nan
B # Same as A's comment

[384,
 214.0,
 427.0,
 50.0,
 364.0,
 269.0,
 350.0,
 407.0,
 253.0,
 96.0,
 179.0,
 97.0,
 99.0,
 116.0,
 88.0,
 137.0,
 104.0,
 23.0,
 80.0,
 83.0,
 72.0,
 88.0,
 258.0,
 218.0,
 465.0,
 725.0,
 101.0,
 112.0,
 11.0,
 717.0,
 772.0,
 150.0,
 503.0,
 256.0,
 780.0,
 200.0,
 799.0,
 22.0,
 15.0,
 117.0,
 24.0,
 128.0,
 165.0,
 198.0,
 1156.0,
 773.0,
 264.0,
 340.0,
 161.0,
 222.0,
 120.0,
 136.0,
 220.0,
 118.0,
 125.0,
 79.0,
 40.0,
 1056.0,
 19.0,
 217.0,
 90.0,
 186.0,
 15.0,
 124.0,
 198.0,
 525.0,
 227.0,
 184.0,
 164.0,
 106.0,
 97.0,
 90.0,
 536.0,
 412.0,
 91.0,
 145.0,
 80.0,
 185.0,
 449.0,
 140.0,
 211.0,
 124.0,
 36.0,
 220.0,
 294.0,
 207.0,
 125.0,
 165.0,
 65.0,
 12.0,
 281.0,
 149.0,
 14.0,
 159.0,
 803.0,
 177.0,
 206.0,
 915.0,
 114.0,
 210.0,
 106.0,
 107.0,
 493.0,
 1275.0,
 172.0,
 207.0,
 219.0,
 263.0,
 97.0,
 380.0,
 84.0,
 214.0,
 567.0,
 18.0,
 11.0,
 148.0,
 293.0,
 342.0,
 64.0,
 145.0,
 168.0,
 68.0,
 253.0,
 59.0,
 103.0,
 146.0,
 636.0,
 227.0,
 13.0,
 2

In [ ]:
print(df1.isnull().sum())

age           295
job            38
marital         0
education     187
default         0
balance       366
housing         0
loan            0
contact      1324
day             0
month           0
duration      291
campaign        0
pdays           0
previous        0
poutcome     3705
y               0
dtype: int64


In [ ]:
df1.head(15)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30.0,unemployed,married,primary,no,1787.0,no,no,cellular,19,oct,79.0,1,-1,0,NaN,no
1,33.0,services,married,secondary,no,4789.0,yes,yes,cellular,11,may,220.0,1,339,4,failure,no
2,35.0,management,single,tertiary,no,1350.0,yes,no,cellular,16,apr,185.0,1,330,1,failure,no
3,30.0,management,married,tertiary,no,1476.0,yes,yes,NaN,3,jun,199.0,4,-1,0,NaN,no
4,59.0,blue-collar,married,secondary,no,0.0,yes,no,NaN,5,may,226.0,1,-1,0,NaN,no
5,35.0,management,single,tertiary,no,747.0,no,no,cellular,23,feb,141.0,2,176,3,failure,no
6,36.0,self-employed,married,tertiary,no,307.0,yes,no,cellular,14,may,341.0,1,330,2,other,no
7,39.0,technician,married,secondary,no,147.0,yes,no,cellular,6,may,151.0,2,-1,0,NaN,no
8,41.0,entrepreneur,married,tertiary,no,221.0,yes,no,NaN,14,may,57.0,2,-1,0,NaN,no
9,43.0,services,married,primary,no,NaN,yes,yes,cellular,17,apr,313.0,1,147,2,failure,no


In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        res = clf.predict(X_train)
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, 
                                                                res)))
        print("Classification Report: \n {}\n".format(classification_report(y_train, 
                                                                            res)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, 
                                                                  res)))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                      lb.transform(res))))

        res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
        print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("\nTest Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, 
                                                                res_test)))
        print("Classification Report: \n {}\n".format(classification_report(y_test, 
                                                                            res_test)))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, 
                                                                  res_test)))   
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), 
                                                      lb.transform(res_test))))
        

**ML Algorithms before Pre-Processing**

In [ ]:
#Splitting Data To Train and Test
y=df1['y']
X = df1.drop(['y'],axis='columns')
y #The one we should predict
X #Inputs basically

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,30.0,unemployed,married,primary,no,1787.0,no,no,cellular,19,oct,79.0,1,-1,0,NaN
1,33.0,services,married,secondary,no,4789.0,yes,yes,cellular,11,may,220.0,1,339,4,failure
2,35.0,management,single,tertiary,no,1350.0,yes,no,cellular,16,apr,185.0,1,330,1,failure
3,30.0,management,married,tertiary,no,1476.0,yes,yes,NaN,3,jun,199.0,4,-1,0,NaN
4,59.0,blue-collar,married,secondary,no,0.0,yes,no,NaN,5,may,226.0,1,-1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33.0,services,married,secondary,no,NaN,yes,no,cellular,30,jul,329.0,5,-1,0,NaN
4517,57.0,self-employed,married,tertiary,yes,NaN,yes,yes,NaN,9,may,153.0,1,-1,0,NaN
4518,57.0,technician,married,secondary,no,295.0,no,no,cellular,19,aug,151.0,11,-1,0,NaN
4519,28.0,blue-collar,married,secondary,no,1137.0,no,no,cellular,6,feb,129.0,4,211,3,other


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test , y_train, y_test =train_test_split(X,y,test_size=0.2)

In [ ]:
#Linear Regression
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
lm = linear_model.LinearRegression()
model = lm.fit(X_train, y_train)

ValueError: ignored

- As we can see it is not even possible to apply any machine learning algorithms when there are null values in the dataset. 
- So first lets do some basic preprocessing like removing rows and then columns that has null values.

In [ ]:
#Droping all the rows with nan values in the dataset 
df_nan_drop_row = df1.dropna(axis=0) # axis = 0 signifies that we are removing rows

In [ ]:
print("Data before droping all the rows with nan values:")
print(df1.shape)
print("Data after droping all the rows with nan values:")
print(df_nan_drop_row.shape)

Data before droping all the rows with nan values:
(4521, 17)
Data after droping all the rows with nan values:
(638, 17)


We can see that many rows has been dropped which is not a good sign or good techinque. So usally we will avoid these situations by applying different preprocessing techniques. But as of now lets deploy any model and measure accuracy score or confusion matrix for the remaining rows.

Well lets start by deploying Logistic regression

In [ ]:
df_nan_drop_row.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [ ]:
logx_data = df_nan_drop_row.drop(['y'],axis=1)#Considering all the rows except the prediction column that is Y
logy_data = df_nan_drop_row['y'] # only the column we need to predict 'y'
logX_train,logX_test,logy_train,logy_test = train_test_split(logx_data,logy_data,test_size = 0.2) #we are spliting the logx_data and logy_data into training and testing data with 20% being the size of test data

In [ ]:
# We are calling the model into the object "log_md"
log_md = LogisticRegression()
#fitting the model to the data
log_md.fit(logX_train,logy_train)

ValueError: ignored

Ohhh seems like we are still facing a problem..... What could that be???  
Machine learning algorithms doesn't work well with categorical values, if you  plan on doing  predictions while there are categorical columns present in the dataset, make sure that those columns are replaced with a number. Like yes/no - 1/0  or male/female - 1/2 etc. Since these are categorical we can always assign a value(number).

We can do this easily by using encoding. Encoding can be done in different ways. So lets explore few of them..


**Editing Below Block**

In [ ]:
#Label encoding for education
for x in df1['education']:
  print(x)

primary
secondary
tertiary
tertiary
secondary
tertiary
tertiary
secondary
tertiary
primary
secondary
secondary
tertiary
secondary
secondary
tertiary
secondary
tertiary
primary
secondary
nan
tertiary
secondary
secondary
tertiary
tertiary
primary
nan
secondary
secondary
secondary
secondary
secondary
tertiary
tertiary
secondary
primary
secondary
secondary
tertiary
tertiary
tertiary
secondary
tertiary
tertiary
tertiary
primary
primary
primary
nan
primary
secondary
secondary
tertiary
secondary
secondary
secondary
tertiary
secondary
secondary
primary
secondary
tertiary
secondary
secondary
secondary
secondary
secondary
secondary
tertiary
tertiary
tertiary
tertiary
tertiary
secondary
tertiary
secondary
secondary
primary
secondary
secondary
primary
secondary
secondary
tertiary
primary
secondary
primary
secondary
secondary
primary
primary
secondary
tertiary
secondary
tertiary
tertiary
tertiary
secondary
primary
secondary
tertiary
secondary
primary
secondary
tertiary
secondary
tertiary
primary
pr